In [1]:
#DATA
#SKU - SKU NUmber
#Std. Price ($) - Price of each unit of SKU
#On-Hand Stock ($) - Total stock on-hand as of 30th Sep'20	 
#APU (units) - Average monthly consumption (30 days) of the SKU
#APU Trend - Anticipated APU trend provided by marketing (e.g. 50% indicates 50% increase in APU consumption)
#S-OTD - Supplier On-time delivery (S-OTD) performance for the past 12 months
#Demand variability (COV) - Demand Coefficient of variance for the past 12 months
#Lead Time (days) - Lead time in days to procure the SKU

#Assumptions to be taken into consideration:
# The Minimum Order Quantity (MOQ) is equal to one lead-time demand
# The service level targeted can be assumed to be 95%
# The expiry for all the mentioned SKUs is one year

#FIND THE FOLLOWING
# Estimate the change in the inventory position to understand the working capital impact  
# Based on the marketing insights gathered, understand the potential obsolete inventory that they currently have

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from scipy import special

In [3]:
#getting data
xls = pd.ExcelFile('2021-ascm-case-competition_school-round_data-exhibits.xlsx')
df = pd.read_excel(xls, 'Data Exhibit-2')

print(list(df.columns))
print(df.dtypes)

['SKU ', 'Std. Price ($)', 'On-Hand Stock ($)', 'APU\n(units)', 'APU Trend', 'S-OTD', 'Demand variability (COV)', 'Lead Time (days)']
SKU                         float64
Std. Price ($)              float64
On-Hand Stock ($)           float64
APU\n(units)                  int64
APU Trend                   float64
S-OTD                       float64
Demand variability (COV)    float64
Lead Time (days)              int64
dtype: object


In [4]:
#Calculate the sum of a Geometric Progression
def sumOfGP(a, r, n):       
    sum = 0
    i = 0
    while i < n: 
        sum = sum + a 
        a = a * r 
        i = i + 1
    return sum

In [5]:
df['Yearly Demand'] = sumOfGP(df['APU\n(units)'] * (1 + df['APU Trend']), 1 + df['APU Trend'], 12)
df['Initial Inventory'] = df['On-Hand Stock ($)'] / df['Std. Price ($)']
df.head()

,SKU,Std. Price ($),On-Hand Stock ($),APU\n(units),APU Trend,S-OTD,Demand variability (COV),Lead Time (days),Yearly Demand,Initial Inventory
0,14171.732125,33.762953,5837.312718,122,-0.1,0.77,1.56,23,7.878924e+02,172.891058
1,52169.042825,12.089326,1532.719476,178,0.2,0.85,1.80,12,8.454395e+03,126.782877
2,72156.257446,11.756134,4562.334137,324,-0.4,0.77,1.38,21,4.849421e+02,388.081145
3,8954.076984,8.157692,4156.895262,414,0.5,0.70,1.99,15,1.599030e+05,509.567542
4,94785.831804,5.269414,587.585764,170,1.2,0.80,0.92,15,4.006164e+06,111.508758


In [11]:
#calculating obsolete inventory
obsolete_inventory = 0
obsolete_sku = pd.DataFrame(np.empty((0, df.shape[1])))    
obsolete_sku.columns = df.columns

total_obsolete_inventory_safety_stock = 0 #Rohan's Insight
#hist_data = []

i = 0
for i in range(len(df)):
    #Finding safety stock of SKUs
    _, _, ss = ROQ(sku_monthly_demand[i], i)
    
    #Condition for obsoletion
    if df.at[i, 'Initial Inventory'] > (df.at[i, 'Yearly Demand'] + ss): 
        obsolete_sku.loc[df.index[i]] = df.iloc[i]
        obsolete_inventory += (df.at[i, 'Initial Inventory'] - df.at[i, 'Yearly Demand'] - ss) * df.at[i, 'Std. Price ($)']
        total_obsolete_inventory_safety_stock += ss * df.at[i, 'Std. Price ($)'] #Rohan's Insight
        #hist_data.append((df.at[i, 'Initial Inventory'] - df.at[i, 'Yearly Demand'] - ss) / ss )
        #n, bins, patches = plt.hist(x=hist_data, bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85)
        #print("Safety Stock of SKU {} is {}".format(i, ss * df.at[i, 'Std. Price ($)']))
        #print("Obsolete Invt. of SKU {} is {}".format(i, (df.at[i, 'Initial Inventory'] - df.at[i, 'Yearly Demand'] - ss) * df.at[i, 'Std. Price ($)']))

#pprint(hist_data)
print("Obsolete Inventory :", obsolete_inventory, "$")
print("KPI || Obsolete Inventory / Safety Stock :", obsolete_inventory/ total_obsolete_inventory_safety_stock * 100, "%") #Rohan's Insight
print(obsolete_sku)

Obsolete Inventory : 825049.5125603355 $
KPI || Obsolete Inventory / Safety Stock : 125.93962016222122 %
              SKU   Std. Price ($)  On-Hand Stock ($)  APU\n(units)  \
9     39145.012382        7.990871       64000.000000         130.0   
19    45569.933059       15.422959        1500.000000          10.0   
37    28494.960963       23.475104       12240.000000          10.0   
41    48429.392292        5.946201        3800.000000          34.0   
52    10035.774733        7.306447         780.000000          34.0   
...            ...             ...                ...           ...   
1859  95224.860171        6.238155       10484.734458        1230.0   
1864  83201.434896        7.462939       18484.796235         906.0   
1867  20959.071060        7.462939       50724.983192        1658.0   
1910  43887.930399        6.769290       89048.395800        2862.0   
1943  13930.053336       16.693611      304565.781899        7414.0   

      APU Trend  S-OTD  Demand variability

In [9]:
#Gathering monthly demand data for all SKUs
sku_monthly_demand = []
for row in range(0, len(df.index)):
    
    #Input GP data
    a = df.iloc[row]['APU\n(units)'] * (1 + df.iloc[row]['APU Trend'])
    r = 1 + df.iloc[row]['APU Trend']
    n = 12
    
    m_demand = []
    total = 0
    
    for i in range(0, n):
        m_demand.append(a*r**(i))
        total += m_demand[i]
    sku_monthly_demand.append(m_demand)

#pprint(sku_monthly_demand)

In [10]:
#Gathering daily demand data for all SKUs
sku_daily_demand = []
for row in range(0, len(df.index)):    
    demand = []
    
    for day in range(0, n*30):
        #Finding month for given day and its corresponding monthly demand
        m = int(day / 30)
        m_d = sku_monthly_demand[row][m]
    
        #ASSUMPTION: Linear Distribution of demand in a month - y = a + x*b
        # a (level) = previous month's last day demand
        if day in range(0, 30): 
            #for initial month a = APU / 30 (ASSUMPTION: Earlier it was Constant Distribution)
            a = df.iloc[row]['APU\n(units)'] / 30
        else:
            a = demand[(m * 30) - 1]
            
        # b (trend) = difference in month end demands / summation of days
        slope = (m_d - 30 * a) / (30 * 31 / 2)
        demand.append(((day+1) - 30 * (m)) * slope + a)
    
    sku_daily_demand.append(demand)

#pprint(sku_daily_demand)

In [23]:
#Calculate ROQ
Z_alpha = 1.645 #as service level is 95% http://www.math.armstrong.edu/statsonline/5/5.3.2.html

def ROQ(LT_demands, row):
    m_d = np.mean(LT_demands)
    s_d = m_d / df.at[row, 'Demand variability (COV)']
    
    m_l = df.at[row, 'Lead Time (days)'] * df.at[row, 'S-OTD']
    x_l = df.at[row, 'Lead Time (days)']     
    
    ##SOURCE: 
        #https://math.stackexchange.com/questions/2560644/calculate-z-score-given-probability-using-erfinv
        #https://www.wolframalpha.com/input/?i=normal+distribution
    ##VERIFICATION:
        #https://stattrek.com/online-calculator/normal.aspx
    z_l = -np.sqrt(2)*special.erfcinv(2*df.at[row, 'S-OTD'])
    s_l = (x_l - m_l)/ z_l
    
    expected_demand = 0
    for i in LT_demands:
        expected_demand += i
        
    safety_stock = Z_alpha * np.sqrt(m_l * s_d**2 + m_d**2 * s_l**2)
    
    return expected_demand + safety_stock, expected_demand, safety_stock

In [81]:
#Calculate the number of days the order lot size shall satisfy demand
def DSD(lot_size, sku, current_day):
    days_satisfied = 0
    while(lot_size > 0):
        try:
            lot_size -= sku_daily_demand[sku][df.at[sku, 'Lead Time (days)'] + current_day + days_satisfied]
        except IndexError:
            break
        days_satisfied += 1
    return days_satisfied - 1

In [93]:
#Final Inventory Levels
number_of_orders = [0] * len(df.index)
lots_ordered = [0] * len(df.index)
day_counter = [0] * len(df.index)
stockout_units = [0] * len(df.index)

inventory_level = [ [0]*360 for _ in range(len(df.index)) ]
arrival = [ [0]*360 for _ in range(len(df.index)) ]

for sku in range(0, len(df.index)):
    inventory_level[sku][0] = df.at[sku, 'Initial Inventory']
    
    for day in range(0, 360):
        #Calculating daily inventory levels of all SKUs
        if day != 0:
            inventory_level[sku][day] = inventory_level[sku][day - 1] - sku_daily_demand[sku][day - 1]
            if (inventory_level[sku][day] < 0 and arrival[sku][day] == 0):
                stockout_units[sku] += abs(inventory_level[sku][day])
                inventory_level[sku][day] = 0
            inventory_level[sku][day] += arrival[sku][day]
            
        if day_counter[sku] == 0:
            #Calculating Lead Time Demands for all SKUs at the given set day
            LT_demands = []
            for time in range(day, day + df.at[sku, 'Lead Time (days)']):
                try:
                    LT_demands.append(sku_daily_demand[sku][time])
                except IndexError:
                    break
            
            #Calculating ReOrder Quantity for the set day
            roq, _, ss = ROQ(LT_demands, sku)
            
            #Order Quantity
            oq = sum(LT_demands) + ss
            
            #Making Ordering Decision
            if inventory_level[sku][day] <= roq:
                number_of_orders[sku] += 1
                try:
                    arrival[sku][day + df.at[sku, 'Lead Time (days)']] += oq
                except IndexError:
                    continue
                lots_ordered[sku] += oq
                day_counter[sku] = DSD(oq, sku, day)
                #day_counter[sku] = df.at[sku, 'Lead Time (days)']
                
        else:
            day_counter[sku] -= 1
            continue

In [94]:
#Checking for Individual Units
i = 1

print("Number of times ordered is : {}".format(number_of_orders[i]))
print("Units Ordered are : {}".format(lots_ordered[i]))
print("Inventory Ordered is : {} $".format(lots_ordered[i] * df.at[i, 'Std. Price ($)']))
print("StockOut Loss is : {} $".format(stockout_units[i] * df.at[i, 'Std. Price ($)']))
print("StockOut Units are : {}".format(stockout_units[i]))
print("Total demand is : {}".format(sum(sku_daily_demand[i])))
print("Service Level is : {} %".format(100 - (stockout_units[i] / sum(sku_daily_demand[i]) * 100)))

Number of times ordered is : 35
Units Ordered are : 9072.083361020936
Inventory Ordered is : 109675.36962172443 $
StockOut Loss is : 376.88277008565854 $
StockOut Units are : 31.174838246201144
Total demand is : 8454.395278737404
Service Level is : 99.6312588042269 %


In [95]:
#KPIs
total_ordering_cost = 0
total_stockout_loss = 0
total_demand = 0

for i in range(len(df.index)):
    total_ordering_cost += lots_ordered[i] * df.at[i, 'Std. Price ($)']
    total_stockout_loss += stockout_units[i] * df.at[i, 'Std. Price ($)']
    total_demand += sum(sku_daily_demand[i]) * df.at[i, 'Std. Price ($)']
    
print("Total Ordering Cost is : {} $".format(total_ordering_cost))
print("Total StockOut Loss is : {} $".format(total_stockout_loss))
print("Total Demand is : {} $".format(total_demand))
print("Total Service Level is : {} %".format(100 - total_stockout_loss / total_demand * 100))

Total Ordering Cost is : 709804372078.8881 $
Total StockOut Loss is : 2568469809.557819 $
Total Demand is : 472066489585.24036 $
Total Service Level is : 99.45590931230588 %


In [96]:
#Debugging Cell
for i in range(len(df.index)):
    if 100 - stockout_units[i]/ sum(sku_daily_demand[i]) * 100 < 95:
        print("SKU {} : Service Level is {} %".format(i, 100 - stockout_units[i]/ sum(sku_daily_demand[i]) * 100))

SKU 105 : Service Level is 70.71513573683563 %
SKU 118 : Service Level is 93.2417499312937 %
SKU 162 : Service Level is 62.270873376700614 %
SKU 305 : Service Level is 93.78035640253037 %
SKU 330 : Service Level is 91.57675879789278 %
SKU 383 : Service Level is 91.52290591606625 %
SKU 403 : Service Level is 93.39163799845464 %
SKU 406 : Service Level is 91.79502241200109 %
SKU 420 : Service Level is 88.23480321202405 %
SKU 423 : Service Level is 71.31707266943722 %
SKU 440 : Service Level is 93.80121073649204 %
SKU 441 : Service Level is 94.04929215003094 %
SKU 446 : Service Level is 86.9420462879077 %
SKU 452 : Service Level is 94.28416945411035 %
SKU 454 : Service Level is 91.5801206284838 %
SKU 456 : Service Level is 86.97112842695215 %
SKU 466 : Service Level is 83.81946924469553 %
SKU 478 : Service Level is 79.58480726468764 %
SKU 483 : Service Level is 93.28044038812718 %
SKU 485 : Service Level is 92.66932015484605 %
SKU 492 : Service Level is 86.47596063641774 %
SKU 496 : Servi

In [98]:
#Inventory Level Analyzer
analyze_sku = 1
ans = pd.DataFrame({'Inventory': inventory_level[analyze_sku], 'Arrival': arrival[analyze_sku], 'Demand': sku_daily_demand[analyze_sku]})
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    print(ans)

      Inventory     Arrival     Demand
0    126.782877    0.000000   6.009892
1    120.772984    0.000000   6.086452
2    114.686533    0.000000   6.163011
3    108.523522    0.000000   6.239570
4    102.283952    0.000000   6.316129
5     95.967823    0.000000   6.392688
6     89.575135    0.000000   6.469247
7     83.105887    0.000000   6.545806
8     76.560081    0.000000   6.622366
9     69.937715    0.000000   6.698925
10    63.238791    0.000000   6.775484
11    56.463307    0.000000   6.852043
12   153.048416  103.437152   6.928602
13   146.119814    0.000000   7.005161
14   139.114652    0.000000   7.081720
15   132.032932    0.000000   7.158280
16   124.874652    0.000000   7.234839
17   117.639814    0.000000   7.311398
18   110.328416    0.000000   7.387957
19   102.940459    0.000000   7.464516
20    95.475943    0.000000   7.541075
21    87.934868    0.000000   7.617634
22    80.317233    0.000000   7.694194
23    72.623040    0.000000   7.770753
24    64.852287    0.0000